**Predicting Sex from Various Crime Tags**

importing containers 

In [25]:
import pandas as pd
import numpy as np
from Clean_Data import data_cleaner
from sklearn import preprocessing


In [26]:
#call columns and change names

data2020 = data_cleaner("Crime_Data_from_2020_to_Present.csv")

data2020.keepColumns(["Rpt Dist No", "Crm Cd", "DATE OCC", "Vict Age", "Vict Descent", "Vict Sex"]) 

oldNames = ["Rpt Dist No", "Crm Cd", "DATE OCC", "Vict Age", "Vict Descent", "Vict Sex"]
newNames = ["District","Code", "Date", "Age", "Descent", "Sex"]
data2020.changeName(oldNames, newNames)

In [27]:
#drop the NaN rows

data2020.delNa()

In [28]:
# drop victims with unknown sex
try:
    data2020.sexToNum()
except TypeError:
    #dont know what this should print cuz the typeerror will already say the error 
    print("There were victim's with unknown sex, will delete all rows of victim's with unknown sex.") 
    data2020.delRows()
    data2020.sexToNum()
finally:
    print("Finished changing victim's sex to numbers (F = 0; M = 1)")

There were victim's with unknown sex, will delete all rows of victim's with unknown sex.
Finished changing victim's sex to numbers (F = 0; M = 1)


In [29]:
data2020.value_counts()

District  Code  Date                    Age  Descent  Sex
1998      310   03/18/2021 12:00:00 AM  35   O        1      20
702       231   05/30/2020 12:00:00 AM  0    W        1      17
723       623   05/30/2020 12:00:00 AM  0    X        1      13
1659      231   08/21/2020 12:00:00 AM  0    W        1      12
702       231   05/30/2020 12:00:00 AM  0    X        1      12
                                                             ..
765       440   08/08/2020 12:00:00 AM  25   A        1       1
                08/05/2021 12:00:00 AM  31   H        1       1
                07/31/2021 12:00:00 AM  27   H        1       1
                07/30/2022 12:00:00 AM  72   B        1       1
2199      745   06/19/2021 12:00:00 AM  49   W        0       1
Length: 468584, dtype: int64